## Jimmy requested eclipse timings for TIC 4707 for a range of days during 2024Q4 so if there's a B-system eclipse we might request that particular day in our proposal.

In [37]:
import astropy.table as tb
from astropy.time import Time
import astropy.units as u


In [5]:
ephemerides = tb.vstack([
    tb.Table.read(r'..\..\django\TargetDB\Data Files\Binary Parameters\Kostov 2022 Binary Parameters.csv'),
    tb.Table.read(r'..\..\django\TargetDB\Data Files\Binary Parameters\Kostov 2023 Binary Parameters.csv'),
])
ephemerides[ephemerides["Local ID"] == "TIC 470710327"]

Local ID,Member,Period,T0 Primary,T0 Secondary,Depth Primary,Depth Secondary,Duration Primary,Duration Secondary
str14,str1,float64,float64,float64,int32,int32,float64,float64
TIC 470710327,A,1.104686,1765.1668,1765.721905,60,50,5.8,5.5
TIC 470710327,B,19.950922,1805.4542,--,70,--,--,--


In [65]:
tess_offset = 2457000
beg_date = Time("2024-11-09", format="iso", scale="utc").jd - tess_offset
end_date = Time("2024-11-21", format="iso", scale="utc").jd - tess_offset

for system in ephemerides[ephemerides["Local ID"] == "TIC 470710327"]:
    for member_letter, member_name in [("a", "Primary"), ("b", "Secondary")]:
        name = f"T0 {member_name}"
        if system[name] > 0:
            t0 = system[name]
            period = system["Period"]
            if not (half_duration := system[f"Duration {member_name}"] / 24 / 2):
                half_duration = 0
            mid_eclipse = t0 + int((beg_date - t0) / period) * period
            while True:
                eclipse_beg = mid_eclipse - half_duration
                eclipse_end = mid_eclipse + half_duration
                print(system["Member"], member_letter, Time(eclipse_beg + tess_offset, format="jd").iso, Time(eclipse_end + tess_offset, format="jd").iso, t0, period)
                mid_eclipse += period
                if mid_eclipse > end_date:
                    break


A a 2024-11-08 15:04:03.533 2024-11-08 20:52:03.533 1765.1668 1.104686
A a 2024-11-09 17:34:48.403 2024-11-09 23:22:48.403 1765.1668 1.104686
A a 2024-11-10 20:05:33.274 2024-11-11 01:53:33.274 1765.1668 1.104686
A a 2024-11-11 22:36:18.144 2024-11-12 04:24:18.144 1765.1668 1.104686
A a 2024-11-13 01:07:03.014 2024-11-13 06:55:03.014 1765.1668 1.104686
A a 2024-11-14 03:37:47.885 2024-11-14 09:25:47.885 1765.1668 1.104686
A a 2024-11-15 06:08:32.755 2024-11-15 11:56:32.755 1765.1668 1.104686
A a 2024-11-16 08:39:17.626 2024-11-16 14:27:17.626 1765.1668 1.104686
A a 2024-11-17 11:10:02.496 2024-11-17 16:58:02.496 1765.1668 1.104686
A a 2024-11-18 13:40:47.366 2024-11-18 19:28:47.366 1765.1668 1.104686
A a 2024-11-19 16:11:32.237 2024-11-19 21:59:32.237 1765.1668 1.104686
A a 2024-11-20 18:42:17.107 2024-11-21 00:30:17.107 1765.1668 1.104686
A b 2024-11-08 02:01:39.734 2024-11-08 07:31:39.734 1765.721905 1.104686
A b 2024-11-09 04:32:24.605 2024-11-09 10:02:24.605 1765.721905 1.104686
A 

In [87]:
def calc_eclipse_timings(
    t0: Time,
    period: u.Quantity,
    duration: u.Quantity = None,
    /,
    beg_time: Time = None,
    end_time: Time = None,
    count: int = 0,
) -> list[tuple[Time, Time]]:
    """Given an initial time (t0), a period and optionally a duration, return a list of eclipses.
    The results can be a fixed number of eclipses, or all eclipses falling between a beginning and end time.
    Period & duration are in days, t0, beg_time & end_time are Astropy Time objects, and count is an int.
    """
    # validate inputs
    if count > 0:
        if beg_time is not None or end_time is not None:
            raise ValueError("Must specify either count or beginning/ending times, not both")
        use_count = True
    else:
        if count < 0:
            raise ValueError("Count cannot be negative")
        if beg_time is None or end_time is None:
            raise ValueError("If count is zero, beginning/ending times must be specified")
        use_count = False

    if duration is not None:
        half_duration = duration / 2
    if use_count:
        t = t0
    else:
        t = t0 + (round((beg_time.jd - t0.jd) / period.value) * period)
    eclipses = []
    while True:
        if duration is not None:
            eclipse_beg = t - half_duration
            eclipse_end = t + half_duration
        else:
            eclipse_beg = t
            eclipse_end = t
        eclipses.append((eclipse_beg, eclipse_end))
        print(eclipse_beg.iso, "   ", eclipse_end.iso)
        t += period
        if use_count:
            if len(eclipses) > count:
                break
        else:
            if end_time < t:
                break
    return eclipses

beg_time=Time("2024-11-09", format="iso", scale="utc")
end_time=Time("2024-11-21", format="iso", scale="utc")

print("Aa")
calc_eclipse_timings(Time(1765.1668 + tess_offset, format="jd"), 1.104686 * u.day, 5.8 * u.hour, beg_time, end_time)
print("Ab")
calc_eclipse_timings(Time(1765.721905 + tess_offset, format="jd"), 1.104686 * u.day, 5.5 * u.hour, beg_time, end_time)
print("Ba")
calc_eclipse_timings(Time(1805.4542 + tess_offset, format="jd"), 19.950922 * u.day, None, beg_time, end_time)


Aa
2024-11-08 15:04:03.533     2024-11-08 20:52:03.533
2024-11-09 17:34:48.403     2024-11-09 23:22:48.403
2024-11-10 20:05:33.274     2024-11-11 01:53:33.274
2024-11-11 22:36:18.144     2024-11-12 04:24:18.144
2024-11-13 01:07:03.014     2024-11-13 06:55:03.014
2024-11-14 03:37:47.885     2024-11-14 09:25:47.885
2024-11-15 06:08:32.755     2024-11-15 11:56:32.755
2024-11-16 08:39:17.626     2024-11-16 14:27:17.626
2024-11-17 11:10:02.496     2024-11-17 16:58:02.496
2024-11-18 13:40:47.366     2024-11-18 19:28:47.366
2024-11-19 16:11:32.237     2024-11-19 21:59:32.237
2024-11-20 18:42:17.107     2024-11-21 00:30:17.107
Ab
2024-11-09 04:32:24.605     2024-11-09 10:02:24.605
2024-11-10 07:03:09.475     2024-11-10 12:33:09.475
2024-11-11 09:33:54.346     2024-11-11 15:03:54.346
2024-11-12 12:04:39.216     2024-11-12 17:34:39.216
2024-11-13 14:35:24.086     2024-11-13 20:05:24.086
2024-11-14 17:06:08.957     2024-11-14 22:36:08.957
2024-11-15 19:36:53.827     2024-11-16 01:06:53.827
2024-1

[(<Time object: scale='utc' format='jd' value=2460620.988102>,
  <Time object: scale='utc' format='jd' value=2460620.988102>)]

## After catching a bug in the one-off code (while testing the above function), do some manual cross-checking to make sure I'm not jerking Jimmy around with bad answers.

In [56]:
foo = Time("2024-11-06 11:42:52.013", format="iso", scale="utc").jd - tess_offset
(foo - 1805.4542) / 19.950922


91.00000000012707

In [58]:
19.950922 * 91 + 1805.4542 + tess_offset

2460620.988102

## Convert CSV file of Kostov ephemerides to something easier to manage, and in standard units.

In [168]:
from astropy.table import QTable

better_ephem = QTable(names=["Local ID", "System", "Member", "T0", "Period", "Duration", "Depth"],
                      dtype=[str, str, str, u.Quantity, u.Quantity, u.Quantity, float])
better_ephem["Period"].unit = u.day
better_ephem["Duration"].unit = u.hour

for ephem in ephemerides:
    for letter, word in [("a", "Primary"), ("b", "Secondary")]:
        if not ephem[f"T0 {word}"] or ephem["Period"] < 0:
            continue
        if ephem[f"Duration {word}"]:
            duration = ephem[f"Duration {word}"] * u.hour
        else:
            duration = None
        better_ephem.add_row([
            ephem["Local ID"],
            ephem["Member"],
            letter,
            Time(ephem[f"T0 {word}"] + tess_offset, format="jd"),
            ephem["Period"] * u.day,
            duration,
            ephem[f"Depth {word}"] / 1000,
            ],
            # mask = [False, False, False, False, False, not (ephem[f"Duration {word}"] > 0), False],
            )

better_ephem.write("Kostov ephemerides.csv", overwrite=True) #[better_ephem["Local ID"] == "TIC 470710327"]
better_ephem

TypeError('The value must be a valid Python or Numpy numeric type.') [astropy.table.table]
TypeError('The value must be a valid Python or Numpy numeric type.') [astropy.table.table]
IndexError('index 0 is out of bounds for axis 0 with size 0') [astropy.table.table]
IndexError('index 0 is out of bounds for axis 0 with size 0') [astropy.table.table]


Local ID,System,Member,T0,Period,Duration,Depth
,,,,d,h,
str14,str1,str1,object,object,object,float64
TIC 9493888,A,a,2458816.2345,2.098992 d,2.9 h,0.146
TIC 9493888,A,b,2458817.283786,2.098992 d,2.7 h,0.117
TIC 25818450,A,a,2458769.9109,10.132402 d,None,0.012
TIC 25818450,A,b,2458776.391584,10.132402 d,None,0.009
TIC 27543409,A,a,2458493.1001,2.122862 d,None,0.05
TIC 27543409,A,b,2458494.153889,2.122862 d,None,0.015
TIC 31928452,A,a,2458337.9129,2.8823 d,2.1 h,0.03
TIC 31928452,A,b,2458339.358085,2.8823 d,1.6 h,0.025


In [176]:
tics_of_interest = [
("2024-05-27", "TIC 344541836"),
("2024-05-28", "TIC 278465736"),
("2024-02-23", "TIC 161043618"),
("2024-02-24", "TIC 367448265"),
]

beg_time = Time("2024-08-23", format="iso", scale="utc")
end_time = Time("2024-09-11", format="iso", scale="utc")

for date, tic_of_interest in tics_of_interest:
    print()
    print(tic_of_interest)
    for ephem in better_ephem[better_ephem["Local ID"] == tic_of_interest]:
        if ephem["Duration"] is None:
            print(f"  {ephem["System"]}{ephem["Member"]}, Period={ephem["Period"]:9.5f}, Duration=unknown, Depth={ephem["Depth"]}")
        else:
            print(f"  {ephem["System"]}{ephem["Member"]}, Period={ephem["Period"]:9.5f}, Duration={ephem["Duration"]:7.4f}, Depth={ephem["Depth"]}")
        calc_eclipse_timings(ephem["T0"], ephem["Period"], ephem["Duration"], beg_time, end_time)


TIC 344541836
  Aa, Period=  2.40993 d, Duration= 3.4000 h, Depth=0.045
2024-08-22 19:41:21.408     2024-08-22 23:05:21.408
2024-08-25 05:31:39.533     2024-08-25 08:55:39.533
2024-08-27 15:21:57.658     2024-08-27 18:45:57.658
2024-08-30 01:12:15.782     2024-08-30 04:36:15.782
2024-09-01 11:02:33.907     2024-09-01 14:26:33.907
2024-09-03 20:52:52.032     2024-09-04 00:16:52.032
2024-09-06 06:43:10.157     2024-09-06 10:07:10.157
2024-09-08 16:33:28.282     2024-09-08 19:57:28.282
  Ab, Period=  2.40993 d, Duration= 4.4000 h, Depth=0.04
2024-08-21 23:06:48.931     2024-08-22 03:30:48.931
2024-08-24 08:57:07.056     2024-08-24 13:21:07.056
2024-08-26 18:47:25.181     2024-08-26 23:11:25.181
2024-08-29 04:37:43.306     2024-08-29 09:01:43.306
2024-08-31 14:28:01.430     2024-08-31 18:52:01.430
2024-09-03 00:18:19.555     2024-09-03 04:42:19.555
2024-09-05 10:08:37.680     2024-09-05 14:32:37.680
2024-09-07 19:58:55.805     2024-09-08 00:22:55.805
2024-09-10 05:49:13.930     2024-09-10

In [162]:
better_ephem[(better_ephem["Local ID"]=="TIC 367448265") & (better_ephem["System"]=="A") & (better_ephem["Member"]=="a")]["Duration"]

-- h
